In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import json
import os
import re
import pandas as pd
import numpy as np

import drive_access
import parsemadzu

re_lcd = re.compile(r"(.*)\.lcd$")
re_txt = re.compile(r"(.*)\.txt$")

files = drive_access.list_files()
print(len(files))

877


In [10]:
def get_path(file: dict) -> str:
    """Find path to file, i.e. its folder location."""
    current_path = ""
    current_file = file
    for i in range(5):
        parents = current_file.get("parents", [])
        parent = parents[0] if parents else None
        
        if parent is None:
            break
        
        parent_search = [f for f in files if f["id"] == parent]
        
        if parent_search is None or len(parent_search) == 0:
            break
        
        parent_file = parent_search[0]
        current_path = os.path.join(parent_file["name"], current_path)
        current_file = parent_file
        
    return current_path

get_path(files[3])

'Project1/2021/10_October/20211004/'

In [11]:
#[f for f in files if "1-LVxSt2qo-5ioi3er3t3QsatEPo9V3-C" in f.get("parents", [])]

rows = []
columns = ["name", "drive_id", "path", "txt_drive_id"]

lcd_files = [f for f in files if re_lcd.match(f["name"])]

for lcd_file in lcd_files:
    txt_file_name = os.path.splitext(lcd_file["name"])[0] + ".txt"
    
    # Find files in same folder with same name but .txt extension instead of .lcd
    txt_file_search = [f for f in files if f["name"] == txt_file_name and f["parents"] == lcd_file["parents"]]

    txt_file = None
    if txt_file_search:
        txt_file = txt_file_search[0]
        
    rows.append([
        lcd_file["name"],
        lcd_file["id"],
        get_path(lcd_file),
        txt_file["id"] if txt_file else np.nan
    ])
    
df = pd.DataFrame(rows, columns=columns).sort_values("name")
df_has_txt = df[np.logical_not(pd.isna(df["txt_drive_id"]))]
df_has_txt

,name,drive_id,path,txt_drive_id
284,20210901 AA GL_01092021_001.lcd,1_PWDxSNzYxkXnHlbxCqxHXibZ59lpSUS,Project1/2021/09_September/20210901/,1p5ZTxdqlJH71DdajNhF83a0SvNRZx4zr
283,20210901 AA GL_01092021_002.lcd,1aj6ei7pTwTZjFw8PNJuNrpuN4HcGFWOe,Project1/2021/09_September/20210901/,1p3PV-Gcot62ARGqNYnyhdw_inTEywlhn
282,20210901 AA GL_01092021_003.lcd,1_qAbDP05iBSZCk6VQukWOkXeOCl1Je_Z,Project1/2021/09_September/20210901/,1p36wr3YTR388msuxEMCQ8esxetZWxUWI
281,20210901 AA GL_01092021_004.lcd,1au8huAEJM40gPUQm52Ll2ze0WuaAPD66,Project1/2021/09_September/20210901/,1pUemLBv7O2ZInvDysN0klCO24MbFB_2n
280,20210901 AA GL_01092021_005.lcd,1a7VS2kPXJayYGQlDu0AHUImL02fDj88a,Project1/2021/09_September/20210901/,1pX9zGM1EHOTMe5xv90KyKskYQ-X2UZKr
...,...,...,...,...
30,20210930 VT_30092021_006.lcd,1Fndh4fymWutFLxQgSFvY94sHJM9vjFt_,Project1/2021/09_September/20210930/,1bgfr8ZObfnACqEM-OnZRn2s1zLinUNUm
29,20210930 VT_30092021_007.lcd,1Frc20sGo7D6J0-pe_k-AWQBmPFe-9IUb,Project1/2021/09_September/20210930/,1bj55lIkH8cXdzFiEscO0rXLHqFRnVvZK
28,20210930 VT_30092021_008.lcd,1GDUsiVC3gNOrP32T1ISyyzafFWjTJ-y2,Project1/2021/09_September/20210930/,1bZKU9LfpeSR3hl-yCBXuzJRLxgwwsf_J
27,20210930 VT_30092021_009.lcd,1GL52WJA05bW-W-az0rx_4iXrKgCbZ5K_,Project1/2021/09_September/20210930/,1bQs_o7r90bHunLq0wQrxURg-Pzqefsrb


In [19]:
def combine_compound_concentrations(df_txt_files: pd.DataFrame) -> pd.DataFrame:
    """Combine a list of result files into a table of concentrations of compounds detected in each experiment."""
    
    rows = []

    for _, row_in in df_txt_files.iterrows():
        f = drive_access.get_file(row_in["txt_drive_id"])
        
        sections = parsemadzu.parse_sections(f)
        compounds = parsemadzu.get_compound_table(sections, detector="B")
        sample_info = parsemadzu.get_sample_information(sections)
        
        row_out = {
            "Acquired Time": sample_info["Acquired"],
            "Dilution Factor": sample_info["Dilution Factor"]
        }
            
        if compounds is None:
            print(f"Skipping {row_in['txt_drive_id']} because of empty compound list")
        else:
            compound_names = compounds["Name"]
            concentrations = compounds["Conc."]

            for compound_name, concentration in zip(compound_names.tolist(), concentrations.tolist()):
                row_out[compound_name] = concentration
        
        rows.append(row_out)

    df_concs = pd.DataFrame(rows)
    df_concs["txt_drive_id"] = df_txt_files["txt_drive_id"].to_numpy()
    
    return df_concs

df_compounds_combined = combine_compound_concentrations(df_has_txt)

Skipping 1oOCuocfnUTXXBM-7ZiHip63AcyZ1uSPS because of empty compound list
Skipping 1oCPuEVyc8NrxWMXHHr684LJ7wAfgCjRD because of empty compound list
Skipping 1oHUiXvNK_cQMYsh2pdZkemEeKayLdh4l because of empty compound list
Skipping 1nwpvArjyx6VmZEk3z9ER3WHyUTz53jwU because of empty compound list
Skipping 1oH0Maq_XJD_WJcobS2L63-KvhVrn0wQ8 because of empty compound list
Skipping 1oEpAEJ_9mEjtxggE6NnOOGNXD3Ufm_k9 because of empty compound list
Skipping 1nrDSOYajY_WTYjIZLlv2qy7a4iKaCOQ_ because of empty compound list
Skipping 1nrfHWtB36jgHrH8OqpBlvik_bDruMSkL because of empty compound list
Skipping 1nnRKVsbyA1QSWOyyJgIp8rKfofHSANYM because of empty compound list
Skipping 1nqvshi__w2ezergHB9ta93XpwklKEGje because of empty compound list
Skipping 1nmvOFTWgMyOjvHgqDbigXfaxZhLmnqVp because of empty compound list
Skipping 1n6HwDsOTUl75JZ1uBlP44O473FJgRq3q because of empty compound list
Skipping 1nD5VzXzEgUUYOfLlPY7tl9sJzsm5kDET because of empty compound list
Skipping 1nE17ckcCKS76GH5ZDS2RzRNOYMfi

In [20]:
df_upload = df_has_txt.merge(df_compounds_combined, on="txt_drive_id")
df_upload.head()

,name,drive_id,path,txt_drive_id,Acquired Time,Dilution Factor,Xylose,Glucose,Mannose,Arabinose,Galactose,Arabitol,Xylitol,Acetic Acid,Glycerol,Ethanol,Citric Acid
0,20210901 AA GL_01092021_001.lcd,1_PWDxSNzYxkXnHlbxCqxHXibZ59lpSUS,Project1/2021/09_September/20210901/,1p5ZTxdqlJH71DdajNhF83a0SvNRZx4zr,01/09/2021 16:01:51,1,0.161,0.0,7.924,0.000,0.89,0.000,0.000,NaN,NaN,NaN,NaN
1,20210901 AA GL_01092021_002.lcd,1aj6ei7pTwTZjFw8PNJuNrpuN4HcGFWOe,Project1/2021/09_September/20210901/,1p3PV-Gcot62ARGqNYnyhdw_inTEywlhn,01/09/2021 17:02:12,1,16.852,0.0,0.953,0.376,0.00,0.559,3.071,NaN,NaN,NaN,NaN
2,20210901 AA GL_01092021_003.lcd,1_qAbDP05iBSZCk6VQukWOkXeOCl1Je_Z,Project1/2021/09_September/20210901/,1p36wr3YTR388msuxEMCQ8esxetZWxUWI,01/09/2021 18:02:34,1,13.672,0.0,0.000,0.345,0.00,0.000,0.000,NaN,NaN,NaN,NaN
3,20210901 AA GL_01092021_004.lcd,1au8huAEJM40gPUQm52Ll2ze0WuaAPD66,Project1/2021/09_September/20210901/,1pUemLBv7O2ZInvDysN0klCO24MbFB_2n,01/09/2021 19:02:55,1,13.618,0.0,0.000,0.341,0.00,0.000,0.000,NaN,NaN,NaN,NaN
4,20210901 AA GL_01092021_005.lcd,1a7VS2kPXJayYGQlDu0AHUImL02fDj88a,Project1/2021/09_September/20210901/,1pX9zGM1EHOTMe5xv90KyKskYQ-X2UZKr,01/09/2021 20:03:16,1,12.772,0.0,0.000,0.349,0.00,0.000,0.000,NaN,NaN,NaN,NaN


## Save to google sheet

In [21]:
import gspread
import df2gspread.df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials


# https://docs.google.com/spreadsheets/d/14DSZKVv0xCJFgxJUDYk-hgTrLtFCkWhZLsexDM3G8L0/edit#gid=0
spreadsheet_id = "14DSZKVv0xCJFgxJUDYk-hgTrLtFCkWhZLsexDM3G8L0"

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
cred = ServiceAccountCredentials.from_json_keyfile_name(
            "secrets/biolabautomation-d5dc83b8ddf7.json", scope)
gc = gspread.authorize(cred)
d2g.upload(df_upload, spreadsheet_id, "autoimport", credentials=cred, row_names=False)

<Worksheet 'autoimport' id:0>